In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchsummary as summary

import torch.nn as nn
import torch.nn.functional as F
import torchsummary as summary

# 0. 하이퍼 파라미터 지정

In [12]:
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Adjust Model Structure Depends on the Data
num_classes = 10 # label 0~9: total 10 classes
in_channel = 3 # black and white images: 1 channel (RGB: 3 channels)

# Hyper-parameters
batch_size = 32
max_pool_kernel = 2
learning_rate = 0.0005
num_epochs = 10

## 1. 데이터 로드

In [13]:
train_data = torchvision.datasets.CIFAR10(root='./datasets',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True) # train set 불러오기
test_data = torchvision.datasets.CIFAR10(root='./datasets',
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True) # test set 불러오기

Files already downloaded and verified
Files already downloaded and verified


# 2. 데이터 로더 정의

In [14]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=True)

# 3. 모델 정의

In [15]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channel, 6, 5, stride=1, padding=2),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(max_pool_kernel)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6, 16, 5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(max_pool_kernel)
    )
    self.fc1 = nn.Linear(8*8*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, num_classes)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = x.reshape(x.size(0),-1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.softmax(self.fc3(x))
    return x

In [16]:
model = ConvNet(num_classes).to(device) # model을 GPU에 올린다

In [17]:
print(model)

# # cf) check the number of parameters
# print('{:=^60}'.format("="))
# print('{:^60}'.format("model summary"))
# print('{:=^60}'.format("="))
# for param_tensor in model.state_dict():
#   print('%-30s'%param_tensor, '{:^30}'.format(str(model.state_dict()[param_tensor].size())))
# print('{:=^60}'.format("="))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# 4. Loss / Optimizer 함수정의

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 5. Train

In [19]:
total_step = len(train_loader) # 1 epoch 당 돌아가는 횟수
loss_list = []

# Train
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Assign Tensors to Configured Device
    images = images.to(device)
    labels = labels.to(device)

    # Forward Propagation
    outputs = model(images)

    # Get Loss, Compute Gradient, Update Parameters
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Append loss to plot graph
    loss_list.append(loss)

    # Print Loss for Tracking Training
    # if (i+1) % 100 == 0 or i == len(train_loader)-1:
    if i == len(train_loader)-1:
      acc = 0
      test_image, test_label = next(iter(test_loader))
      _, test_predicted = torch.max(model(test_image.to(device)).data, 1)

      for (pred, ans) in zip(test_predicted, test_label):
        if pred == ans:
          acc += 1
      acc = acc / len(test_predicted)

      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.1f}%'.format(epoch+1,num_epochs, i+1, total_step, loss.item(), acc*100))
      # print('Testing data: [Predicted: {} / Real: {}]'.format(test_predicted, test_label))

  if epoch+1 == num_epochs:
    torch.save(model.state_dict(), 'model.pth')
  else:
    torch.save(model.state_dict(), 'model-{:02d}_epochs.pth'.format(epoch+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/10], Step [1563/1563], Loss: 2.0532, Accuracy: 53.1%
Epoch [2/10], Step [1563/1563], Loss: 1.9405, Accuracy: 53.1%
Epoch [3/10], Step [1563/1563], Loss: 1.6878, Accuracy: 65.6%
Epoch [4/10], Step [1563/1563], Loss: 1.7801, Accuracy: 53.1%
Epoch [5/10], Step [1563/1563], Loss: 1.9638, Accuracy: 59.4%
Epoch [6/10], Step [1563/1563], Loss: 1.8842, Accuracy: 62.5%
Epoch [7/10], Step [1563/1563], Loss: 1.9470, Accuracy: 50.0%
Epoch [8/10], Step [1563/1563], Loss: 1.8086, Accuracy: 59.4%
Epoch [9/10], Step [1563/1563], Loss: 1.9563, Accuracy: 65.6%
Epoch [10/10], Step [1563/1563], Loss: 1.6532, Accuracy: 59.4%


In [20]:
# Test after training is done
model.eval() # Set model as evaluation mode (instead of mini-batch mean/var, batchnorm is used)
with torch.no_grad(): # auto_grad off
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the {} test images {}%'.format(len(test_loader)*batch_size, 100*correct/total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10016 test images 62.43%
